In [88]:
import findspark
findspark.init()

from pyspark.sql import SQLContext
import pyspark

sc = pyspark.SparkContext(appName="Calories")
sqlContext = SQLContext(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Calories, master=local[*]) created by __init__ at <ipython-input-1-5b7e48e25840>:7 

In [706]:
fs_source = "/home/up/Documents/databases/"
fs_output = "/home/up/Documents/output/"

creditcard_df = (sqlContext.read.format("com.databricks.spark.csv").options(header=True, inferSchema=True, delimiter=",")
                 .load(fs_source+"ehresp_2014.csv"))

In [707]:
creditcard_df.show(1)

+--------------+--------+---------+---------+------+--------+--------+--------+-------+-----+-----+----------+-------+-----+----------+--------+--------+-----------+--------+-------+---------+------+--------+--------+-----+--------+---------+------+------+--------+------+--------+----------+-------+-----+-----+---------+
|      tucaseid|tulineno|eeincome1|    erbmi|erhhch|erincome|erspemch|ertpreat|ertseat|ethgt|etwgt|eudietsoda|eudrink|eueat|euexercise|euexfreq|eufastfd|eufastfdfrq|euffyday|eufdsit|eufinlwgt|eusnap|eugenhth|eugroshp|euhgt|euinclvl|euincome2|eumeat|eumilk|euprpmel|eusoda|eustores|eustreason|eutherm|euwgt|euwic|exincome1|
+--------------+--------+---------+---------+------+--------+--------+--------+-------+-----+-----+----------+-------+-----+----------+--------+--------+-----------+--------+-------+---------+------+--------+--------+-----+--------+---------+------+------+--------+------+--------+----------+-------+-----+-----+---------+
|20140101140007|       1|      

In [708]:
creditcard_df.printSchema()

root
 |-- tucaseid: long (nullable = true)
 |-- tulineno: integer (nullable = true)
 |-- eeincome1: integer (nullable = true)
 |-- erbmi: double (nullable = true)
 |-- erhhch: integer (nullable = true)
 |-- erincome: integer (nullable = true)
 |-- erspemch: integer (nullable = true)
 |-- ertpreat: integer (nullable = true)
 |-- ertseat: integer (nullable = true)
 |-- ethgt: integer (nullable = true)
 |-- etwgt: integer (nullable = true)
 |-- eudietsoda: integer (nullable = true)
 |-- eudrink: integer (nullable = true)
 |-- eueat: integer (nullable = true)
 |-- euexercise: integer (nullable = true)
 |-- euexfreq: integer (nullable = true)
 |-- eufastfd: integer (nullable = true)
 |-- eufastfdfrq: integer (nullable = true)
 |-- euffyday: integer (nullable = true)
 |-- eufdsit: integer (nullable = true)
 |-- eufinlwgt: double (nullable = true)
 |-- eusnap: integer (nullable = true)
 |-- eugenhth: integer (nullable = true)
 |-- eugroshp: integer (nullable = true)
 |-- euhgt: integer (nulla

In [724]:
#Removendo colunas que não serão inicialmente usadas
temp_df = (creditcard_df.drop("tucaseid").drop("tulineno")
           .drop("erhhch").drop("eeincome1").drop("exincome1").drop("euincome2").drop("erspemch").drop("ethgt")
            .drop("etwgt").drop("euffyday").drop("eufdsit").drop("eusnap").drop("eugenhth").drop("eugroshp")
            .drop("euhgt").drop("euinclvl").drop("euprpmel").drop("eustores").drop("eustreason")
            .drop("eutherm").drop("euwgt").drop("euwic").drop("ertpreat").drop("eufinlwgt")
            .drop("erincome"))

In [725]:
temp_df.printSchema()

root
 |-- erbmi: double (nullable = true)
 |-- ertseat: integer (nullable = true)
 |-- eudietsoda: integer (nullable = true)
 |-- eudrink: integer (nullable = true)
 |-- eueat: integer (nullable = true)
 |-- euexercise: integer (nullable = true)
 |-- euexfreq: integer (nullable = true)
 |-- eufastfd: integer (nullable = true)
 |-- eufastfdfrq: integer (nullable = true)
 |-- eumeat: integer (nullable = true)
 |-- eumilk: integer (nullable = true)
 |-- eusoda: integer (nullable = true)



In [726]:
temp_df.select("*").show(1)

+---------+-------+----------+-------+-----+----------+--------+--------+-----------+------+------+------+
|    erbmi|ertseat|eudietsoda|eudrink|eueat|euexercise|euexfreq|eufastfd|eufastfdfrq|eumeat|eumilk|eusoda|
+---------+-------+----------+-------+-----+----------+--------+--------+-----------+------+------+------+
|33.200001|      2|        -1|      2|    1|         2|      -1|       2|         -1|     1|     2|    -1|
+---------+-------+----------+-------+-----+----------+--------+--------+-----------+------+------+------+
only showing top 1 row



In [727]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import IntegerType

temp_df = temp_df.where("erbmi>=1")


def to_zero(x):
    if x < 0 :
        return -1
    else:
        return x

udf = UserDefinedFunction(lambda x: to_zero(x), IntegerType())

temp_df = temp_df.withColumn('ertseat1', udf(temp_df.ertseat))
temp_df = temp_df.withColumn('eudietsoda1', udf(temp_df.eudietsoda))
temp_df = temp_df.withColumn('eudrink1', udf(temp_df.eudrink))
temp_df = temp_df.withColumn('eueat1', udf(temp_df.eueat))
temp_df = temp_df.withColumn('euexercise1', udf(temp_df.euexercise))
temp_df = temp_df.withColumn('euexfreq1', udf(temp_df.euexfreq))
temp_df = temp_df.withColumn('eufastfdfrq1', udf(temp_df.eufastfdfrq))
temp_df = temp_df.withColumn('eumeat1', udf(temp_df.eumeat))
temp_df = temp_df.withColumn('eumilk1', udf(temp_df.eumilk))
temp_df = temp_df.withColumn('eusoda1', udf(temp_df.eusoda))
temp_df = temp_df.withColumn('eufastfd1', udf(temp_df.eufastfd))

temp_df = (temp_df
           .drop('ertseat')
            .drop('eudietsoda')
            .drop('eudrink')
            .drop('eueat')
            .drop('euexercise')
            .drop('euexfreq')
            .drop('eufastfdfrq')
            .drop('eumeat')
            .drop('eumilk')
            .drop('eusoda')
           .drop('eufastfd'))

In [728]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import FloatType

temp_df = temp_df.where("erbmi>=1")

def modify_values(erbmi):
    if erbmi >= 25:
        return 1.0
    else:
        return 0.0

udf = UserDefinedFunction(lambda x: modify_values(x), FloatType())

temp_df = temp_df.withColumn('target_column', udf(temp_df.erbmi))

In [729]:
temp_df=temp_df.drop("erbmi")

In [730]:
# Dividindo os dados em treinamento e teste
weights = [0.9, 0.1]
seed = 13579

training_df, test_df = temp_df.randomSplit(weights, seed)
training_df.cache()
test_df.cache()

DataFrame[ertseat1: int, eudietsoda1: int, eudrink1: int, eueat1: int, euexercise1: int, euexfreq1: int, eufastfdfrq1: int, eumeat1: int, eumilk1: int, eusoda1: int, eufastfd1: int, target_column: float]

DataFrame[ertseat1: int, eudietsoda1: int, eudrink1: int, eueat1: int, euexercise1: int, euexfreq1: int, eufastfdfrq1: int, eumeat1: int, eumilk1: int, eusoda1: int, eufastfd1: int, target_column: float]

In [731]:
print("Número de linhas do Dataframe de treinamento: {0}".format(training_df.count()))
print("Número de linhas do Dataframe de teste: {0}".format(test_df.count()))
print("Distribuicao de fraudes no Dataframe de treinamento: {0}".format(training_df.groupBy("target_column").count().take(4)))

Número de linhas do Dataframe de treinamento: 9603
Número de linhas do Dataframe de teste: 1034
Distribuicao de fraudes no Dataframe de treinamento: [Row(target_column=1.0, count=6187), Row(target_column=0.0, count=3416)]


In [732]:
# Efetuando as transformações para gerar features e preparar dados para passagem ao indutor
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

features = training_df.columns[0:10]

feature_assembler = VectorAssembler(inputCols=features, outputCol="features")
label_indexer = StringIndexer(inputCol="target_column", outputCol="label")
rf = RandomForestClassifier(labelCol='label', featuresCol='features', numTrees=3, 
                            featureSubsetStrategy="auto", impurity="gini", maxDepth=4, maxBins=32, seed=seed)

instance = [feature_assembler, label_indexer, rf]
pipeline = Pipeline(stages=instance)

In [733]:
model_df = pipeline.fit(training_df)
transformed = model_df.transform(test_df)

In [734]:
#transformed.select("erbmi", "prediction").show(50)
transformed.printSchema()

root
 |-- ertseat1: integer (nullable = true)
 |-- eudietsoda1: integer (nullable = true)
 |-- eudrink1: integer (nullable = true)
 |-- eueat1: integer (nullable = true)
 |-- euexercise1: integer (nullable = true)
 |-- euexfreq1: integer (nullable = true)
 |-- eufastfdfrq1: integer (nullable = true)
 |-- eumeat1: integer (nullable = true)
 |-- eumilk1: integer (nullable = true)
 |-- eusoda1: integer (nullable = true)
 |-- eufastfd1: integer (nullable = true)
 |-- target_column: float (nullable = true)
 |-- features: vector (nullable = true)
 |-- label: double (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



In [735]:
transformed.select("prediction","label").orderBy("prediction", "label").show(10000);

+----------+-----+
|prediction|label|
+----------+-----+
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|

In [736]:
trasformed_new = (
    transformed.select(
        "*",
        transformed.prediction.cast('string').alias('target_string'),
        transformed.label.cast('string').alias('label_string'))
)

In [737]:
# Gerando métricas dos dados [TIRAR]
from pyspark.mllib.evaluation import BinaryClassificationMetrics as metric

results_rfc = trasformed_new.select(['target_string', 'label_string'])
results_collect = results_rfc.collect()

results_list = [(float(i[0][0]), 1.0-float(i[1])) for i in results_collect]
scoreAndLabels = sc.parallelize(results_list)

metrics = metric(scoreAndLabels)
print("Área sob a curva de Precision/Recall (PR): {0:.2f}".format(metrics.areaUnderPR * 100))
print("O ROC score para 3 arvores: {0:.2f}".format(metrics.areaUnderROC * 100))

Área sob a curva de Precision/Recall (PR): 57.63
O ROC score para 3 arvores: 49.14


In [738]:
# Gerando métricas adicionais
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

predAndLabel_mlp = transformed.select(['prediction', 'label'])

accuracy_rfc = MulticlassClassificationEvaluator(metricName="accuracy")
f1_rfc = MulticlassClassificationEvaluator(metricName="f1")

print ("Accuracy MLP: {0}".format(accuracy_rfc.evaluate(predAndLabel_mlp)))
print ("F1-Score MLP: {0}".format(f1_rfc.evaluate(predAndLabel_mlp)))

Accuracy MLP: 0.6450676982591876
F1-Score MLP: 0.5287212726985008
